
# Теория Кодирования
## Лабораторная работа 6

От студента группы **6405**-010302
**Слеповичева Святослава**


---

### Задание
#### 6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом g(x)=1+x2+x3, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

In [92]:
import numpy as np
from sympy import Symbol, div

In [93]:
def encode(a, g, n):
    a_polynom = 0
    for idx, el in enumerate(a):
        a_polynom += el * x**idx

    coefficient_dict = dict((a_polynom * g).expand().as_coefficients_dict().items())
    received_word = []
    for i in range(n):
        received_word.append((coefficient_dict.setdefault(x**i, 0)) % 2)
    return received_word


n, k = 7, 4

x = Symbol('x')
g = 1 + x**2 + x**3

word = np.array([1, 0, 0, 1])
encoded_word = encode(word, g, n)

---
#### 6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом g(x)=1+x3+x4+x5+x6, исправляющего пакеты ошибок кратности 3

In [94]:
def decode(a, g, n, k: int, t: int):
    a_polynom = 0
    for idx, el in enumerate(a):
        a_polynom += el * x**idx

    r = div(a_polynom, g)[1]
    coefficient_dict_cicle = dict(r.as_coefficients_dict().items())
    syndrome = 0
    for i in range(n):
        syndrome += ((coefficient_dict_cicle.setdefault(x**i, 0) % 2) * x**i)

    for i in range(n):
        r = div(x**i * syndrome, g)[1]
        coefficient_dict_cicle = dict(r.as_coefficients_dict().items())
        syndrome_i = 0
        for j in range(n):
            syndrome_i += ((coefficient_dict_cicle.setdefault(x**j, 0) % 2) * x**j)

        if sum(syndrome_i.as_coefficients_dict().values()) <= t:
            e = (x**(n - i) * syndrome_i).expand()
            result_polynom = a_polynom + e

            coefficient_dict_cicle = dict(result_polynom.as_coefficients_dict().items())
            result_polynom = 0
            for l in range(n):
                result_polynom += ((coefficient_dict_cicle.setdefault(x**l, 0)) % 2) * x**l

            coefficient_dict_cicle = dict(div(result_polynom, g)[0].as_coefficients_dict().items())
            result_word = []
            for l in range(k):
                result_word.append((coefficient_dict_cicle.setdefault(x**l, 0)) % 2)

            return np.array(result_word, dtype=int)
    return None


---
#### И провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4.

In [95]:
# Однократно ошиблись
broken_word = encoded_word.copy()
broken_word[0] ^= 1

all(decode(broken_word, g, n, k, 1) == word)

True

In [96]:
# Двукратно ошиблись
broken_word = encoded_word.copy()
broken_word[2] ^= 1
broken_word[3] ^= 1

all(decode(broken_word, g, n, k, 2) == word)

False

In [97]:
# Трёхкратно ошиблись
broken_word = encoded_word.copy()
broken_word[2] ^= 1
broken_word[3] ^= 1
broken_word[4] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

False

---
#### И для пакетов ошибок

In [98]:
g = 1 + x ** 3 + x ** 4 + x ** 5 + x ** 6
n, k = 15, 9


# Один раз ошиблись
word = np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])
encoded_word = encode(word, g, n)
broken_word = encoded_word.copy()

broken_word[5] ^= 1

all(decode(broken_word, g, n, k, 1) == word)

True

In [99]:
# Второй раз ошиблись
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[7] ^= 1

all(decode(broken_word, g, n, k, 2) == word)

True

In [100]:
# Третий раз ошиблись
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[6] ^= 1
broken_word[7] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

False

In [101]:
# Четвёртый раз ошиблись
broken_word = encoded_word.copy()

broken_word[5] ^= 1
broken_word[6] ^= 1
broken_word[7] ^= 1
broken_word[8] ^= 1

all(decode(broken_word, g, n, k, 3) == word)

False